In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

In [ ]:
!ls -al /kaggle/input

In [2]:
#raw_data = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-train.csv')
raw_data = pd.read_csv('jigsaw-toxic-comment-train.csv')

In [3]:
raw_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
tf.__version__

'2.2.0'

In [6]:
#https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [86]:
#raw_mini = raw_data[:10000]
raw_mini = raw_data

In [13]:
raw_mini.groupby('toxic').count()
#shows class imbalance. 90% of comments are non toxic.
#SMOTE or ADASYN can be used to over sample minority class. TODO
#https://imbalanced-learn.org/stable/over_sampling.html#from-random-over-sampling-to-smote-and-adasyn

,id,comment_text,severe_toxic,obscene,threat,insult,identity_hate
toxic,,,,,,,
0,90388,90388,90388,90388,90388,90388,90388
1,9612,9612,9612,9612,9612,9612,9612


In [218]:
def get_batch(X, size = 10000, clean = False):
    lenx = len(X)
    start = 0
    end = min(size, lenx)
    while end<=lenx and start != end:
        #print(f"start {start}, end {end}")
        X_temp = X['comment_text'][start:end]
        if clean:
            X_temp = X.map(clean_txt)
        retX = embed(X_temp)
        rety = X['toxic'][start:end]
        yield retX, rety
        start = end
        end = min(end+size, lenx)


In [75]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [102]:
#X_train, X_test, y_train, y_test = train_test_split(embeddings_mini.numpy(), raw_mini['toxic'], test_size=0.3, random_state=0)
def test_train(df, test_ratio = 0.3):
    idx_test = random.sample(range(len(df)), int(test_ratio*len(df)))
    idx_train = np.setdiff1d(range(len(df)), idx_test)
    raw_test = df.iloc[idx_test, :]
    raw_train = df.iloc[idx_train, :]
    return raw_test, raw_train

In [103]:
raw_test, raw_train = test_train(raw_mini)

## Logistic Regression

In [89]:
logreg = SGDClassifier(loss='log') 

batch = 0

for X, y in get_batch(raw_train, size = 10000):
    logreg.partial_fit(X, y, classes = [0,1])
    print(str(batch)+" ", end="")
    batch += 1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

In [107]:
X_test = embed(raw_test['comment_text'])
y_test = raw_test['toxic']
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.94


In [108]:
y_pred = logreg.predict(X_test)

In [109]:
confusion_matrix(y_test, y_pred)

array([[59821,   760],
       [ 2977,  3506]])

In [110]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     60581
           1       0.82      0.54      0.65      6483

    accuracy                           0.94     67064
   macro avg       0.89      0.76      0.81     67064
weighted avg       0.94      0.94      0.94     67064



## SVM Classifier

In [111]:
svc = SGDClassifier()

batch = 0

for X, y in get_batch(raw_train, size = 10000):
    svc.partial_fit(X, y, classes = [0,1])
    print(str(batch)+" ", end="")
    batch += 1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

In [112]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(svc.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.93


In [115]:
y_pred = svc.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[60453,   128],
       [ 4403,  2080]])

In [116]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     60581
           1       0.94      0.32      0.48      6483

    accuracy                           0.93     67064
   macro avg       0.94      0.66      0.72     67064
weighted avg       0.93      0.93      0.92     67064



## Oversampling Minority class

In [117]:
from sklearn.utils import resample

In [120]:
raw_train_minority = raw_train[raw_train['toxic']==1]

In [127]:
#How many majority class samples are there?
len(raw_train)-len(raw_train_minority)
#141584

141584

In [129]:
raw_train_minority_upsample = resample(raw_train_minority, 
                                       replace=True,
                                       n_samples=141584,
                                       random_state=1)

In [131]:
raw_train_balanced = pd.concat([raw_train[raw_train['toxic']==0], raw_train_minority_upsample])

In [144]:
raw_train_balanced = raw_train_balanced.sample(frac=1)

In [145]:
raw_train_balanced.groupby('toxic')['id'].count()

toxic
0    141584
1    141584
Name: id, dtype: int64

In [146]:
logreg1 = SGDClassifier(loss='log') 

batch = 0

for X, y in get_batch(raw_train_balanced, size = 10000):
    logreg1.partial_fit(X, y, classes = [0,1])
    print(str(batch)+" ", end="")
    batch += 1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

In [147]:
y_pred = logreg1.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[53555,  7026],
       [  769,  5714]])

In [148]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93     60581
           1       0.45      0.88      0.59      6483

    accuracy                           0.88     67064
   macro avg       0.72      0.88      0.76     67064
weighted avg       0.93      0.88      0.90     67064



## A closer look at the errors

In [159]:
raw_test[(y_test == 1) & (y_test != y_pred)]['comment_text']

52738     fok yu 2 \n\nYOU OLSOW SUK AS MUZIKANIMUL FOK ...
30798     Please ban me \n\nThis pedia is a wiki pedia a...
129964                          == \nFUCK YOU DAVE SOUZA ==
7985      [edit] Oh Man...you deserve it!!!\n  The RickK...
161804    :If you'd address the problem, then I wouldn't...
                                ...                        
177054    == E kala mai (my apologies) == \n\n Aloha, MZ...
28272     "\n\nHow is it the wrong way round? If I had f...
25945     BNP aren't fascist\n\nStop trolling the BNP ar...
152131    ":::John Strange's own nutjob words: ""But in ...
85060     Ian Black - Caley \n\nHave you seen Ian Black ...
Name: comment_text, Length: 769, dtype: object

In [222]:
raw_test['comment_text'][85060]

'Ian Black - Caley \n\nHave you seen Ian Black play. His irractic and life endagering challenges on Dunfermline and Rangers players, show he is an actual fucking nutter. He was when he played for the Blackburn youth team and he still is.'

In [161]:
raw_test['comment_text'][30798]

'Please ban me \n\nThis pedia is a wiki pedia and not an encyclo pedia of useful knowledge instead it gives mis information which acts to pro mote agendas'

In [162]:
raw_test['comment_text'][129964]

'== \nFUCK YOU DAVE SOUZA =='

In [163]:
raw_test['comment_text'][161804]

":If you'd address the problem, then I wouldn't feel it necessary to be so blunt. John fucked up again. Have you told him that?"

#### Are there comments with f*** word that are not toxic?
##### Yes.

In [172]:
raw_test[(raw_test['comment_text'].str.contains('fuck|Fuck|FUCK')) & (raw_test['toxic']==0)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
176842,4496f798bef5aaf1,""" \n :And yet no evidence whatsoever that any ...",0,0,0,0,0,0
205636,b799019c26ded5f5,"Niamh Byrne is a single parent , after getting...",0,0,0,0,0,0
166824,1d31e742b354abd7,""" \n :I'm very much an agnostic. I didn't oppo...",0,0,0,0,0,0
167104,1e710c75803affce,"From where I sit, it summarizes the career of ...",0,0,0,0,0,0
185443,66f44152e46d9d56,"""Is this supposed to be frightening? Oh no! Wi...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
209583,c745d2e66ad200af,Welcome to MTV idea of reality. This is why m...,0,0,0,0,0,0
187975,71364b0da53ab591,""" \n\n :Hi and thanks for your suggestion! ...",0,0,0,0,0,0
96471,03ff64306c7cec32,"""\n\nPOV and OR\n\nI have removed what appears...",0,0,0,0,0,0
193800,88bc869e74c52268,: Frank Joseph has been a Nazi a VERY LONG TIM...,0,0,0,0,0,0


In [173]:
raw_test['comment_text'][176842]

'" \n :And yet no evidence whatsoever that any one of 19 million American cannabis users has ever been admitted to the hospital for MI due to their cannabis use.  None.  But you\'ll continue to cite sources sans evidence that use the unscientific fear-mongering term ""marijuana"" instead of the medical term ""cannabis"" because those NIDA dollars have to keep rolling in and the DEA must meet their quota. Pure fucking nonsense.  You\'re no skeptic, that\'s for sure.   "'

In [174]:
raw_test['comment_text'][205636]

"Niamh Byrne is a single parent , after getting knocked up , she contacted MTV for them to do a Dublin Style version of 16 and Pregnant , they told her to fuck off and get off the streets in your pajamas...dirty bitch. Her local junkie boyfriend , Rasher left Niamh on the news that she was pregnant , he left her to become a drug mule and maybe later even promoted to a gansTA. \n She is currently living in a box in Seagrange Park because she got kicked out of her house when she held her fish up for ransom , for money to buy drugs. \n Niamhs hopes and goals for her future is for her to become the local Cat Lady who walks around with her stolen trolly from Lidl throwing her cats at people. \n We're right behind you Niamh every step of the way , Do us proud or the barrel out of first year with get mr burrrrke after you."

#### Do characters like \n and = affect embeddings?
##### Yes they do.

In [177]:
tryclean = embed(['To be or not to be. That is the question.'])
trydirty1 = embed(['\n\n To be or not to be. That is the question.'])
trydirty2 = embed(['== To be or not to be. That is the question.'])

In [192]:
def cosd(a, b):
    return np.dot(a[0], b[0])/(np.linalg.norm(a[0])*np.linalg.norm(b[0]))

In [194]:
cosd(tryclean, trydirty1)

0.8699262

In [195]:
cosd(tryclean, trydirty2)

0.7407079

In [196]:
cosd(trydirty1, trydirty2)

0.79356325

## Things to try next
#### Clean up data. For example like this - https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb


In [201]:
import re

In [202]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text) #non word characters
    text = re.sub('\s+', ' ', text) #whitespace characters
    text = text.strip(' ')
    return text

In [212]:
txt1 = "This is a normal text with 123 and \n\n"

In [219]:
clean_text(txt1)

'this is a normal text with 123 and'

In [220]:
cosd(embed([txt1]), embed([clean_text(txt1)]))

0.9057607

## Logistic Regression with clean text

In [221]:
logreg = SGDClassifier(loss='log') 

batch = 0

for X, y in get_batch(raw_train, size = 10000, clean=True):
    logreg.partial_fit(X, y, classes = [0,1])
    print(str(batch)+" ", end="")
    batch += 1

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
X_test = embed(raw_test['comment_text'])
y_test = raw_test['toxic']

In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
y_pred = logreg.predict(X_test)

confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

### Things to try
##### DL architecture
##### LASER embedding
##### RoBERTa embedding